<a href="https://colab.research.google.com/github/koushik395/Web-scraping-Hotel-Data-from-booking.com/blob/main/WebScraping_Hotel_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web Scraping Data From Booking.com using python and BeautifulSoup

### Import the neccessary Modules for scraping the Data

In [37]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import re
import datetime as dt

#### Enter the required Details of journey Date and other relevant information for scraping the Data.

In [38]:
isValid= True

while isValid:
  try:
    start_date , start_month , start_year = list(map(str,input("Enter Journey Start Date (dd-mm-yy): ").split('-')))
    end_date , end_month , end_year = list(map(str,input("Enter Journey End Date (dd-mm-yy): ").split('-')))
    dt.datetime(int(start_year),int(start_month),int(start_date))
    dt.datetime(int(end_year),int(end_month),int(end_date))
    isValid=False
  except ValueError:
     print("Invalid Date!!!")

Destination = input("Enter your Destination Place: ")
adults , children , no_rooms =list(map(str,input("Enter no. of adults-children-no.of.rooms: ").split('-')))

Enter Journey Start Date (dd-mm-yy): 19-01-2023
Enter Journey End Date (dd-mm-yy): 21-01-2023
Enter your Destination Place: Mumbai
Enter no. of adults-children-no.of.rooms: 4-2-2


#### Functions to retrieve the Details from the website.

In [39]:
def get_Hotel_Name(soup):

    try:
        # Outer Tag Object
        title = soup.find("h2", attrs={"class":'d2fee87262 pp-header__title'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_Hotel_Rating(soup):
    try:
        # Outer Tag Object
        title = soup.find("div", attrs={"class":'b5cd09854e d10a6220b4'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_Room_type(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"class":"hprt-roomtype-icon-link"})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string


def get_Price(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"class":"prco-valign-middle-helper"})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_Room_Sqft(soup):
    try:
        # Outer Tag Object
        title = soup.find("div", attrs={"data-name-en":'room size'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_Hotel_Facilities(soup):
    try:
        new_list =[]
        ## Getting the outer div tags
        for item in soup.find_all("div", attrs={"class":"ec680921b4"}):
          ## For every div search for the facilitites div and extract the text
          new_list.append(item.find("div",attrs={"class":"db29ecfbe2"}).text.strip())

    except AttributeError:
       new_list=""

    return new_list

def get_Hotel_Location(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"data-node_tt_id":"location_score_tooltip"})

         # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string



#### Starter code to verify whether we are connected to the Website or Not.

In [ ]:
if __name__ == '__main__':
  HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

  URL = "https://www.booking.com/searchresults.html?aid=304142&label=gen173rf-1FCAEoggI46AdIM1gDaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGiAg1wcm9qZWN0cHJvLmlvqAIDuAKwwPadBsACAdICJDU0NThkNDAzLTM1OTMtNDRmOC1iZWQ0LTdhOTNjOTJmOWJlONgCBeACAQ&sid=2214b1422694e7b065e28995af4e22d9&sb=1&sb_lp=1&src=theme_landing_index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fhotel%2Findex.html%3Faid%3D304142%26label%3Dgen173rf-1FCAEoggI46AdIM1gDaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGiAg1wcm9qZWN0cHJvLmlvqAIDuAKwwPadBsACAdICJDU0NThkNDAzLTM1OTMtNDRmOC1iZWQ0LTdhOTNjOTJmOWJlONgCBeACAQ%26sid%3D2214b1422694e7b065e28995af4e22d9%26&ss=goa&is_ski_area=0&checkin_year=2023&checkin_month=1&checkin_monthday=13&checkout_year=2023&checkout_month=1&checkout_monthday=14&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&offset=0"

  webpage = requests.get(URL,headers=HEADERS)
  
  soup = BeautifulSoup(webpage.content,"html.parser")

  print(soup)

#### Main code to iterate through every Page of the website and retrieve the neccessary information about the hotels.

In [41]:
page_no=0
d = {"Hotel_Name":[], "Hotel_Rating":[], "Room_type":[],"Room_price":[],"Room_sqft":[],"Facilities":[],"Location":[]}
while (page_no<=50):
     URL = f"https://www.booking.com/searchresults.html?aid=304142&label=gen173rf-1FCAEoggI46AdIM1gDaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGiAg1wcm9qZWN0cHJvLmlvqAIDuAKwwPadBsACAdICJDU0NThkNDAzLTM1OTMtNDRmOC1iZWQ0LTdhOTNjOTJmOWJlONgCBeACAQ&sid=2214b1422694e7b065e28995af4e22d9&sb=1&sb_lp=1&src=theme_landing_index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fhotel%2Findex.html%3Faid%3D304142%26label%3Dgen173rf-1FCAEoggI46AdIM1gDaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGiAg1wcm9qZWN0cHJvLmlvqAIDuAKwwPadBsACAdICJDU0NThkNDAzLTM1OTMtNDRmOC1iZWQ0LTdhOTNjOTJmOWJlONgCBeACAQ%26sid%3D2214b1422694e7b065e28995af4e22d9%26&ss={Destination}&is_ski_area=0&checkin_year={start_year}&checkin_month={start_month}&checkin_monthday={start_date}&checkout_year={end_year}&checkout_month={end_month}&checkout_monthday={end_date}&group_adults={adults}&group_children={children}&no_rooms={no_rooms}&b_h4u_keep_filters=&from_sf=1&offset={page_no}"
     new_webpage = requests.get(URL, headers=HEADERS)
     soup = BeautifulSoup(new_webpage.content,"html.parser")
     links = soup.find_all("a", attrs={"class":"e13098a59f"})
     for link in links:
        new_webpage = requests.get(link.get('href'), headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        d["Hotel_Name"].append(get_Hotel_Name(new_soup))
        d["Hotel_Rating"].append(get_Hotel_Rating(new_soup))
        d["Room_type"].append(get_Room_type(new_soup))
        d["Room_price"].append(get_Price(new_soup))
        d["Room_sqft"].append(get_Room_Sqft(new_soup))
        d["Facilities"].append(get_Hotel_Facilities(new_soup))
        d["Location"].append(get_Hotel_Location(new_soup))

     page_no += 25

#### Converting the Resultant dictionary to Pandas DataFrame.

#### We have Successfully Scraped the Data from the Booking.com Website.

In [43]:
Hotel_df = pd.DataFrame.from_dict(d)
Hotel_df.to_csv("Hotel_data.csv", header=True, index=False)
print(Hotel_df.head())

                                  Hotel_Name Hotel_Rating  \
0                  Novotel Mumbai Juhu Beach          6.7   
1                      Trident Nariman Point          8.8   
2       Novotel Mumbai International Airport          7.8   
3  OYO Flagship 80682 Hotel Golden Residency                
4                     Hotel Ariana Residency          7.2   

                            Room_type Room_price  Room_sqft  \
0  Superior Twin Room with Ocean View       $785  312 feet²   
1              Trident Preferred Room       $748  301 feet²   
2                  Standard King Room       $537  357 feet²   
3                 Classic Triple Room       $130              
4                  Deluxe Double Room       $294  200 feet²   

                                          Facilities  \
0  [Free parking, Pet friendly, Swimming pool, Re...   
1  [Free parking, Swimming pool, Restaurant, Spa,...   
2  [Swimming pool, Free WiFi, Free parking, Air c...   
3  [Air conditioning, Private 